Embeddings

In [ ]:
# !python -m spacy download en_core_web_lg

In [1]:
from embedding_utils import *

/Users/olinewcombe/Documents/GitHub/LLMWorkshop/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
doc = nlp("dog cat banana apple")

similarities = np.zeros((4, 4))

for i, token1 in enumerate(doc):
    for j, token2 in enumerate(doc):
        similarities[i, j] = token1.similarity(
            token2
        )  # computes the cosine similarity

pd.DataFrame(
    similarities,
    index=["Dog", "Cat", "Banana", "Apple"],
    columns=["Dog", "Cat", "Banana", "Apple"],
)

,Dog,Cat,Banana,Apple
Dog,1.000000,0.822082,0.209090,0.228810
Cat,0.822082,1.000000,0.223588,0.203681
Banana,0.209090,0.223588,1.000000,0.664670
Apple,0.228810,0.203681,0.664670,1.000000


In [3]:
init_notebook_mode(connected=True)
words = ["cat", "meow", "dog", "woof", "bird", "tweet", "lion", "roar", "horse", "neigh", ]

pca = PCA()

glove_vectors = np.concatenate(
    [nlp(word).vector.reshape(1, 300) for word in words]
)

glove_pca = pca.fit_transform(glove_vectors)
scatter(glove_pca[:, 0], glove_pca[:, 1], np.array(words), np.array(words))

## Exercises
1) Think about why these relationships aren't exact (here it isn't exactly king - man + woman = queen). I can think of two main ones. For one - why may tweet be a bit of an outlier here?
2) Play around yourself and find some relationships - really try and think about whats going on here - what are the dimensions each representing? 

As a note, investigating what it is AI 'understands' is a hard problem, but there are ways to gain intuition. I suggest Semantle (https://semantle.com/). They use a different embedding: word2vec, brilliantly visualised by TensorFlow (https://projector.tensorflow.org/).

# Case Study - Newspost Data

In [4]:
df = pd.read_csv('data/newsposts/newsposts_science.csv', index_col=0)
labels = df['Class Name'].astype('category').values.unique().tolist()
df.head()

,Text,Class Name
0,(Graham Toal)Re: Do we need the clipper for c...,sci.crypt
1,(Graham Toal)Let's build software cryptophone...,sci.crypt
2,"(technopagan priest)Re: Would ""clipper"" make ...",sci.crypt
3,"(Vesselin Bontchev)Re: Once tapped, your code...",sci.crypt
4,"(Phil G. Fraering)Re: Once tapped, your code ...",sci.crypt


In [5]:
df['glove_embedding'] = df['Text'].apply(get_embedding_glove)

In [7]:
init_notebook_mode(connected=True)

X = np.vstack(df['glove_embedding'].values)
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

hover_plot(X_pca, df)

We see poor separation.

In [ ]:
df['SBERT_embedding'] = df['Text'].apply(get_embedding_SBERT)

In [13]:
X = np.vstack(df['SBERT_embedding'].values)

tsne = PCA(n_components=2)
X_tsne = tsne.fit_transform(X)

hover_plot(X_tsne, df)

We are now starting to see true separation, but, can we do better? What can we change?

## Exercise:

Change the dimensionality reduction technique. Try t-sne.

In [14]:
X = np.vstack(df['SBERT_embedding'].values)

tsne = TSNE(n_components=2)
X_tsne = tsne.fit_transform(X)

hover_plot(X_tsne, df)

T-sne does come with the disadvantage of lost interpretability

## Extra Exercises

Try UMAP - this is likely the most commonly used in practice as it has the best performance, but is rather complicated.

Perform k-means on this two dimensional plot and do your own test/train split to see how effective this embedding method is.